In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

22/03/01 16:38:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Question 1
pyspark.__version__

'3.0.3'

In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-28 18:55:36--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.88.228
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.88.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  31.8MB/s    in 23s     

2022-02-28 18:55:59 (30.6 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [26]:
# Pandas is used to get the schema
import pandas as pd

In [6]:
df_pandas = pd.read_csv('fhvhv_tripdata_2021-02.csv', nrows=100)

In [7]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [30]:
from pyspark.sql import types

In [12]:
schema = types.StructType([
    types.StructField("hvfhs_license_num",types.StringType(),True),
    types.StructField("dispatching_base_num",types.StringType(),True),
    types.StructField("pickup_datetime",types.TimestampType(),True),
    types.StructField("dropoff_datetime",types.TimestampType(),True),
    types.StructField("PULocationID",types.IntegerType(),True),
    types.StructField("DOLocationID",types.IntegerType(),True),
    types.StructField("SR_Flag",types.StringType(),True)
])

In [13]:
df_spark = spark.read \
    .schema(schema) \
    .option("header", "true") \
    .csv("fhvhv_tripdata_2021-02.csv")

In [17]:
df_spark

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [14]:
df_spark.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [16]:
df_spark.repartition(24).write.parquet('data/pq/hvfhv/2021/02/')

In [18]:
df_spark.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [12]:
from pyspark.sql import functions as F

In [25]:
df_spark = df_spark \
    .withColumn('pickup_date', F.to_date(df_spark.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df_spark.dropoff_datetime))

In [26]:
df_spark.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null| 2021-02-01|  2021-02-01|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null| 2021-02-01|  2021-02-01|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null| 2021-02-01|  2021-02-01|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null| 2021-02-01|  2021-02-01|
|           HV0003| 

In [42]:
# Question 3 using CSV data

df_spark \
    .select('pickup_date') \
    .filter(df_spark.pickup_date == '2021-02-15') \
    .count()

367170

In [3]:
# Reading the parquet data for the remaining questions:
df = spark.read.parquet('data/pq/hvfhv/2021/02/')

In [14]:
df = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))

In [46]:
df.registerTempTable('hvfhv_tripdata')

In [16]:
# Question 4
spark.sql("""
SELECT
    pickup_date,
    UNIX_TIMESTAMP(dropoff_datetime) - UNIX_TIMESTAMP(pickup_datetime) AS trip_duration
FROM
    hvfhv_tripdata
ORDER BY
    trip_duration DESC
""").show()

+-----------+-------------+
|pickup_date|trip_duration|
+-----------+-------------+
| 2021-02-11|        75540|
| 2021-02-17|        57221|
| 2021-02-20|        44039|
| 2021-02-03|        40653|
| 2021-02-19|        37577|
| 2021-02-25|        35010|
| 2021-02-20|        34806|
| 2021-02-18|        34612|
| 2021-02-18|        34555|
| 2021-02-10|        34169|
| 2021-02-10|        32476|
| 2021-02-25|        32439|
| 2021-02-21|        32223|
| 2021-02-09|        32087|
| 2021-02-06|        31447|
| 2021-02-02|        30913|
| 2021-02-10|        30856|
| 2021-02-09|        30732|
| 2021-02-21|        30660|
| 2021-02-05|        30511|
+-----------+-------------+
only showing top 20 rows



In [17]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, pickup_date: date, dropoff_date: date]

In [24]:
df \
    .groupBy('dispatching_base_num') \
    .count() \
    .sort(F.desc('count')) \
    .show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [25]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-03-01 17:24:54--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.73.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.73.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-03-01 17:24:55 (89.9 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [28]:
df_pandas = pd.read_csv('taxi+_zone_lookup.csv', nrows=100)

In [29]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(LocationID,LongType,true),StructField(Borough,StringType,true),StructField(Zone,StringType,true),StructField(service_zone,StringType,true)))

In [31]:
schema = types.StructType([
    types.StructField("LocationID",types.IntegerType(),True),
    types.StructField("Borough",types.StringType(),True),
    types.StructField("Zone",types.StringType(),True),
    types.StructField("service_zone",types.StringType(),True)
])

In [32]:
df_zones = spark.read \
    .schema(schema) \
    .option("header", "true") \
    .csv("taxi+_zone_lookup.csv")

In [33]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [47]:
df_zones.registerTempTable('zones')

In [52]:
spark.sql("""
SELECT
    CONCAT(zp.Zone, '/', zd.Zone) AS pickup_dropoff_pair,
    COUNT(1) as count
FROM
    hvfhv_tripdata ht
LEFT JOIN zones zp ON
    ht.PULocationID = zp.LocationID
LEFT JOIN zones zd ON
    ht.DOLocationID = zd.LocationID
GROUP BY
    pickup_dropoff_pair
ORDER BY
    count DESC
""").show()

+--------------------+-----+
| pickup_dropoff_pair|count|
+--------------------+-----+
|East New York/Eas...|45041|
|Borough Park/Boro...|37329|
|   Canarsie/Canarsie|28026|
|Crown Heights Nor...|25976|
| Bay Ridge/Bay Ridge|17934|
|Jackson Heights/J...|14688|
|     Astoria/Astoria|14688|
|Central Harlem No...|14481|
|Bushwick South/Bu...|14424|
|Flatbush/Ditmas P...|13976|
|South Ozone Park/...|13716|
|Brownsville/Brown...|12829|
|      JFK Airport/NA|12542|
|Prospect-Lefferts...|11814|
|Forest Hills/Fore...|11548|
|Bushwick North/Bu...|11491|
|Bushwick South/Bu...|11487|
|Crown Heights Nor...|11462|
|Crown Heights Nor...|11342|
|Prospect-Lefferts...|11308|
+--------------------+-----+
only showing top 20 rows

